In [1]:
import pandas as pd
import pandas.io.sql as sqlio
import pprint as pp
import psycopg2
import sqlalchemy
import os
import json
import uuid

In [2]:
conn = psycopg2.connect(
    user="dcat", 
    password="dcat", 
    host="127.0.0.1", 
    port="54320", 
    database="dcat")

In [3]:
cur = conn.cursor()

In [4]:
print ( conn.get_dsn_parameters(),"\n")

{'user': 'dcat', 'dbname': 'dcat', 'host': '127.0.0.1', 'port': '54320', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'gssencmode': 'prefer', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 



In [5]:
cur.execute("SELECT version();")
record = cur.fetchone()
print("You are connected to - ", record,"\n")

You are connected to -  ('PostgreSQL 12.1 (Debian 12.1-1.pgdg100+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 8.3.0-6) 8.3.0, 64-bit',) 



In [6]:
cur.execute("CREATE EXTENSION IF NOT EXISTS pgcrypto;")

In [7]:
conn.commit()

In [8]:
cur.execute("drop index if exists idx_tbl_edge_n1")
cur.execute("drop index if exists idx_tbl_edge_n2")
cur.execute("drop table if exists tbl_edge")
cur.execute("drop table if exists tbl_node")

In [9]:
cur.execute("""
create table tbl_node 
(  
  id SERIAL PRIMARY KEY,
  prop JSONB,
  schema VARCHAR(500),
  prop_id VARCHAR(500),
  type VARCHAR(500),
  valid_from TIMESTAMP default now(),
  valid_to TIMESTAMP,
  valid BOOLEAN default FALSE
);  
""")

In [10]:
sql = "select * from tbl_node"
df = sqlio.read_sql_query(sql, conn)
df.head()

,id,prop,schema,prop_id,type,valid_from,valid_to,valid


In [11]:
conn.commit()
sql = 'select * from pg_catalog.pg_user'
df_users = sqlio.read_sql_query(sql, conn)

In [12]:
cur.execute("drop table if exists tbl_edge")
cur.execute("""
create table tbl_edge     -- What is relationship between ID1 to ID2      
(  
  n1 int references tbl_node(id),    
  n2 int references tbl_node(id),
  prop JSONB, 
  valid_from TIMESTAMP default now(),
  valid_to TIMESTAMP,
  valid BOOLEAN, 
  check (n1<>n2),  
  unique (n1,n2)  
);  
""")

In [13]:
sql = "select * from tbl_edge"
df = sqlio.read_sql_query(sql, conn)
df.head()

,n1,n2,prop,valid_from,valid_to,valid


In [14]:
cur.execute("drop index if exists idx_tbl_edge_n1")
cur.execute("create index idx_tbl_edge_n1 on tbl_edge(n1)")

In [15]:
cur.execute("drop index if exists idx_tbl_edge_n2")
cur.execute("create index idx_tbl_edge_n2 on tbl_edge(n2)")

In [16]:
conn.commit()

### Populate table

In [17]:
cur.execute("SELECT version();")
record = cur.fetchone()
print("You are connected to - ", record,"\n")

You are connected to -  ('PostgreSQL 12.1 (Debian 12.1-1.pgdg100+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 8.3.0-6) 8.3.0, 64-bit',) 



### Test Data

In [18]:
test_data = [{
    "name": "test_prop",
    "schema": "test_schema",
    "type": "test_type"
},{
    "name": "test_prop2",
    "schema": "test_schema2",
    "type": "test_type2" 
},{
    "name": "test_prop3",
    "schema": "test_schema3",
    "type": "test_type3" 
}]

In [19]:
sql="""insert into tbl_node (prop, schema,prop_id,type,valid) 
    values"""

for index in test_data:
    sql= sql + f""" ('{json.dumps(index)}','{index['schema']}','{index['name']}','{index['type']}', TRUE),"""


sql=sql[:-1]
cur.execute(sql)
conn.commit()

In [20]:
df = sqlio.read_sql_query("select * from tbl_node", conn)
df

,id,prop,schema,prop_id,type,valid_from,valid_to,valid
0,1,"{'name': 'test_prop', 'type': 'test_type', 'sc...",test_schema,test_prop,test_type,2020-01-07 11:41:45.636181,None,True
1,2,"{'name': 'test_prop2', 'type': 'test_type2', '...",test_schema2,test_prop2,test_type2,2020-01-07 11:41:45.636181,None,True
2,3,"{'name': 'test_prop3', 'type': 'test_type3', '...",test_schema3,test_prop3,test_type3,2020-01-07 11:41:45.636181,None,True


In [21]:
test_update = f"""UPDATE tbl_node set valid_to=now(), valid=FALSE 
    where valid_to is Null AND prop_id IN ("""

for index in test_data:
    test_update = test_update + f""" '{index['name']}',"""
    
test_update=test_update[:-1]
test_update=test_update+");"

cur.execute(test_update)
conn.commit()

In [22]:
df = sqlio.read_sql_query("select * from tbl_node", conn)
df

,id,prop,schema,prop_id,type,valid_from,valid_to,valid
0,1,"{'name': 'test_prop', 'type': 'test_type', 'sc...",test_schema,test_prop,test_type,2020-01-07 11:41:45.636181,2020-01-07 11:41:47.064147,False
1,2,"{'name': 'test_prop2', 'type': 'test_type2', '...",test_schema2,test_prop2,test_type2,2020-01-07 11:41:45.636181,2020-01-07 11:41:47.064147,False
2,3,"{'name': 'test_prop3', 'type': 'test_type3', '...",test_schema3,test_prop3,test_type3,2020-01-07 11:41:45.636181,2020-01-07 11:41:47.064147,False
